<a href="https://colab.research.google.com/github/Yousuf-Basir/Facebook-event-scraper/blob/main/image_superres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Image Super-Resolution

Colab for [superresolution_gan](https://github.com/fukumame/superresolution_gan), implementation of [Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network](https://arxiv.org/abs/1609.04802).

Seems to work best on very small images. Images larger than 200x200 (in total number of pixels) will be sliced, processed in tiles and combined.

In [2]:
#@title Mount Drive
from google.colab import drive
drive.mount('/content/drive')
drive_root = "/content/drive/My Drive/"

!pip -q install import-ipynb
!pip -q install image_slicer
!curl -q -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Settings

In [14]:
#@title Path to an image file in Google Drive
input_image = "/content/drive/MyDrive/ai/superres/doggo-goodboy.png" #@param {type:"string"}
input_image = drive_root+input_image

#@markdown <font color="#999">Output will be saved in the same directory as `superres_<filename>`</font>

dir_out = path_dir(input_image)
output_image = path_dir(input_image)+"superres_"+path_leaf(input_image)

target_grid  = 200*200

dir_input_slices = dir_tmp+"input-slices/"
dir_output_slices = dir_tmp+"output-slices/"
create_dirs([dir_tmp, dir_input_slices, dir_output_slices])

### Setup

In [8]:
#@title Git clone & get model
%cd /content/
!git clone {git_q} https://github.com/fukumame/superresolution_gan.git
%cd /content/superresolution_gan
!wget {wget_q} https://www.dropbox.com/s/l4s5a6v4licks62/generator_model_3008000.npz
import image_slicer, math
from PIL import Image

/content
fatal: destination path 'superresolution_gan' already exists and is not an empty directory.
/content/superresolution_gan


### **Action**

In [15]:
#@title Superres
im = Image.open(input_image)
width, height = im.size

# Tile & save tiles
def slice_image(input_image, dir):
  tiles = closestDiv(math.floor((width*height)/target_grid), 4)
  slicery = image_slicer.slice(input_image, tiles, save=False)
  image_slicer.save_tiles(slicery, directory=dir, prefix='slice')

# Superres tiles
def superres(dir_in, dir_out):
  slices = glob(dir_in+"slice*")
  %cd /content/superresolution_gan/
  i = 1
  for slice in slices:
    output = dir_out+path_leaf(slice)
    print('> processing slice', str(i)+"/"+str(len(slices)))
    !python superresolution.py --modelpath=generator_model_3008000.npz --imagepath="{slice}" --outputpath="{output}"
    i+=1

# Mash
def mash(dir_in, img_out):
  tiles = image_slicer.open_images_in(dir_in)
  image = image_slicer.join(tiles)
  tmpImg = dir_tmp+rnd_str(8)
  image.save('/content/tmp/joined.png')
  !cp "/content/tmp/joined.png" "{output_image}"

# Cleanup
def clean():
  copy = dir_tmp+path_leaf(input_image)
  !rm {dir_input_slices}*
  !rm {dir_input_slices}*
  !rm {dir_tmp}joined.png
  !rm {copy}

def closestDiv(a, b):
  c1 = a - (a % b)
  c2 = (a + b) - (a % b)
  if (a - c1 > c2 - a):
    return c2
  else:
    return c1

if width*height > target_grid:
  !cp "{input_image}" "{dir_tmp}"
  print('Slice', input_image+'...')
  slice_image(input_image, dir_input_slices)
  print('Done.\nSuperres slices...')
  superres(dir_input_slices, dir_output_slices)
  print('Done.\nMash back into single image...')
  mash(dir_output_slices, output_image)
  print('.Done\nFile saved as', output_image)
  clean()
else:
  %cd /content/superresolution_gan
  print('Superres', input_image)
  !python superresolution.py --modelpath=generator_model_3008000.npz --imagepath="{input_image}" --outputpath="{output_image}"
  print('Saved to', output_image)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive//content/drive/MyDrive/ai/superres/doggo-goodboy.png'